<a href="https://colab.research.google.com/github/danielhcg/AntiSmishGPT/blob/main/startOverGemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import google.generativeai as genai
import os
from google.colab import userdata
import pandas as pd
import time

In [3]:
api_key = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=api_key)

model = genai.GenerativeModel('models/gemini-1.5-pro')

In [24]:
# proof of concept to make sure api is working
# only takes in a sinle message
def detect(message):
    prompt = f"Is this text message real or fake? '{message}' Respond with one word: true, false, or undetermined."
    response = model.generate_content(prompt)
    return response.text.strip()

# Example usage
message = "Your account has been suspended. Please click the link to reactivate it."
result = detect(message)

print(result)

Undetermined


In [27]:
# takes in entire spreadsheet and spits out new spreadsheet
# doesn't consider rate limits
# works for spreadsheet with one entry
def detect(message):
    prompt = f"Is this text message real or fake? '{message}' Respond with one word: true, false, or undetermined."
    response = model.generate_content(prompt)
    return response.text.strip()

# Define the input file path
input_file = 'mendeley_set.xlsx'  # Update with your file path

# Load the data into a pandas DataFrame without headers
df = pd.read_excel(input_file, header=None)

# Assume the messages are in the first column
df['authenticity'] = df[0].apply(detect)

# Save the updated results back to the same file
df.to_excel(input_file, index=False, header=False)

print("Processing complete. Results saved to", input_file)

Processing complete. Results saved to mendeley_set.xlsx


In [34]:
# takes in entire spreadsheet and spits out new spreadsheet
# considers rate limit
import time

# Function to process a single message
def detect(message):
    prompt = f"Is this text message real or fake? '{message}' Respond with one word: true, false, or undetermined."
    response = model.generate_content(prompt)
    return response.text.strip()

# Function to process the Excel file
def process_excel(input_file):
    # Load the data into a pandas DataFrame without headers
    df = pd.read_excel(input_file, header=None)

    # To store results for each value
    authenticity_results = []

    # Determine the number of tokens per message
    def count_tokens(text):
        return len(text.split())

    # Calculate tokens for each message
    df['tokens'] = df[0].apply(count_tokens)

    # Process rows in batches considering token limits
    batch_size = 20  # Only one message per request to keep it simple
    max_requests_per_day = 20
    max_tokens_per_minute = 32000
    max_requests_per_minute = 2

    requests_made = 0
    total_tokens_processed = 0

    for i in range(len(df)):
        if requests_made >= max_requests_per_day:
            print("Daily limit reached. Stopping execution.")
            break

        message = df[0][i]
        tokens = df['tokens'][i]

        if total_tokens_processed + tokens > max_tokens_per_minute:
            print("Token limit per minute reached. Waiting for the next minute.")
            time.sleep(60)  # Wait for the next minute
            total_tokens_processed = 0  # Reset the token count for the new minute

        result = detect(message)
        authenticity_results.append(result)
        total_tokens_processed += tokens

        # Print status after processing each message
        print(f"Processed message {i + 1}/{len(df)}, Result = {result}")

        # Rate limiting: ensure only 2 requests per minute
        requests_made += 1
        if requests_made % max_requests_per_minute == 0:
            time.sleep(60)

    # Assign results to new column
    df['authenticity'] = authenticity_results[:len(df)]

    # Save the updated results back to the same file
    output_file = "processed_" + input_file
    df.to_excel(output_file, index=False, header=False)

    print("Processing complete. Results saved to", output_file)

# Example usage
input_file = 'mendeley_set.xlsx'  # Update with your file path
process_excel(input_file)

Processed message 1/40, Result = Undetermined
Processed message 2/40, Result = Undetermined
Processed message 3/40, Result = Undetermined
Processed message 4/40, Result = Undetermined
Processed message 5/40, Result = false
Processed message 6/40, Result = false
Processed message 7/40, Result = undetermined
Processed message 8/40, Result = undetermined


KeyboardInterrupt: 

In [15]:
# to check what models are available
import pprint
for model in genai.list_models():
    pprint.pprint(model)

Model(name='models/chat-bison-001',
      base_model_id='',
      version='001',
      display_name='PaLM 2 Chat (Legacy)',
      description='A legacy text-only model optimized for chat conversations',
      input_token_limit=4096,
      output_token_limit=1024,
      supported_generation_methods=['generateMessage', 'countMessageTokens'],
      temperature=0.25,
      top_p=0.95,
      top_k=40)
Model(name='models/text-bison-001',
      base_model_id='',
      version='001',
      display_name='PaLM 2 (Legacy)',
      description='A legacy model that understands text and generates text as an output',
      input_token_limit=8196,
      output_token_limit=1024,
      supported_generation_methods=['generateText', 'countTextTokens', 'createTunedTextModel'],
      temperature=0.7,
      top_p=0.95,
      top_k=40)
Model(name='models/embedding-gecko-001',
      base_model_id='',
      version='001',
      display_name='Embedding Gecko',
      description='Obtain a distributed representatio

In [37]:
#processed sheet, but output was incorrect

# Function to process a batch of messages
def detect(messages):
    prompts = [f"Is this text message real or fake? '{message}' Respond with one word: true, false, or undetermined." for message in messages]
    responses = model.generate_content(prompts)  # Assuming the model can handle a list of prompts
    return [response.text.strip() for response in responses]

# Function to process the Excel file
def process_excel(input_file):
    # Load the data into a pandas DataFrame without headers
    df = pd.read_excel(input_file, header=None)

    # To store results for each value
    authenticity_results = []

    # Determine the number of tokens per message
    def count_tokens(text):
        return len(text.split())

    # Calculate tokens for each message
    df['tokens'] = df[0].apply(count_tokens)

    # Process rows in batches of 20 messages
    batch_size = 20
    max_requests_per_day = 50
    max_tokens_per_minute = 32000
    max_requests_per_minute = 2

    requests_made = 0
    total_tokens_processed = 0

    for i in range(0, len(df), batch_size):
        if requests_made >= max_requests_per_day:
            print("Daily limit reached. Stopping execution.")
            break

        batch = df[0][i:i+batch_size].tolist()
        batch_tokens = df['tokens'][i:i+batch_size].sum()

        if total_tokens_processed + batch_tokens > max_tokens_per_minute:
            print("Token limit per minute reached. Waiting for the next minute.")
            time.sleep(60)  # Wait for the next minute
            total_tokens_processed = 0  # Reset the token count for the new minute

        batch_results = detect(batch)
        authenticity_results.extend(batch_results)
        total_tokens_processed += batch_tokens

        # Print status after processing each batch
        print(f"Processed inputs {i + 1} to {i + len(batch)}, Results = {batch_results}")

        # Rate limiting: ensure only 2 requests per minute
        requests_made += 1
        if requests_made % max_requests_per_minute == 0:
            time.sleep(60)

    # Assign results to new column
    df['authenticity'] = pd.Series(authenticity_results)

    # Save the updated results back to the same file
    output_file = "processed_" + input_file
    df.to_excel(output_file, index=False, header=False)

    print("Processing complete. Results saved to", output_file)

# Example usage
input_file = 'mendeley_set.xlsx'  # Update with your file path
process_excel(input_file)

Processed inputs 1 to 20, Results = ['Undetermined\nUndetermined\nUndetermined\nUndetermined\nFalse\nFalse\nUndetermined\nUndetermined\nUndetermined\nUndetermined\nUndetermined\nUndetermined\nUndetermined\nUndetermined\nUndetermined\nFalse\nUndetermined\nTrue\nUndetermined\nUndetermined']
Processed inputs 21 to 40, Results = ["Here's a breakdown of each text message and whether they seem real or fake:\n\n* **'(That said can you text him one more time?)'** - **Undetermined** (Could be real, depends on context)\n* **'I know she called me'** - **Undetermined** (Could be real, depends on context) \n* **'In sch but neva mind u eat 1st lor..'** - **Undetermined** (Uses slang/shortening, could be real)\n* **'lyricalladie(21/F) is inviting you to be her friend. Reply YES-910 or NO-910. See her: www.SMS.ac/u/hmmross STOP? Send STOP FRND to 62468'** - **False** (Classic spam format)\n* **'Jay says that you're a double-faggot'** - **Undetermined** (Could be real, but uses offensive language)\n* *

In [6]:
#most up to date
#handles everything

# Function to process a batch of messages
def detect(messages):
    prompts = [f"Is this text message real or fake? '{message}' Respond with one word: Real, Fake, or Undetermined, without any explanations." for message in messages]
    response = model.generate_content(prompts)

    # Debugging: Print response object to understand its structure
    print(f"Response: {response}")

    # Extract text from the response and split
    response_text = response.candidates[0].content.parts[0].text.strip()
    split_results = response_text.split('\n')

    # Further filter results to ensure only valid responses are considered
    valid_responses = ['true', 'false', 'undetermined']
    results = [line.strip().lower() for line in split_results if line.strip().lower() in valid_responses]

    # Debugging: Print filtered results
    print(f"Filtered results: {results}")

    return results

# Function to process the CSV file
def process_csv(input_file):
    # Load the data into a pandas DataFrame without headers
    df = pd.read_csv(input_file, header=None)

    # To store results for each value
    authenticity_results = []

    # Determine the number of tokens per message
    def count_tokens(text):
        return len(text.split())

    # Calculate tokens for each message
    df['tokens'] = df[0].apply(count_tokens)

    # Process rows in batches of 20 messages
    batch_size = 20
    max_requests_per_day = 50
    max_tokens_per_minute = 32000
    max_requests_per_minute = 2

    requests_made = 0
    total_tokens_processed = 0

    for i in range(0, len(df), batch_size):
        if requests_made >= max_requests_per_day:
            print("Daily limit reached. Stopping execution.")
            break

        batch = df[0][i:i+batch_size].tolist()
        batch_tokens = df['tokens'][i:i+batch_size].sum()

        if total_tokens_processed + batch_tokens > max_tokens_per_minute:
            print("Token limit per minute reached. Waiting for the next minute.")
            time.sleep(60)  # Wait for the next minute
            total_tokens_processed = 0  # Reset the token count for the new minute

        try:
            batch_results = detect(batch)
        except Exception as e:
            print(f"Error during detection: {e}")
            batch_results = ['undetermined'] * batch_size  # Fallback to undetermined for all in case of error

        if len(batch_results) != batch_size:
            print(f"Error: Expected {batch_size} results but got {len(batch_results)}. Continuing with next batch.")
            batch_results += ['undetermined'] * (batch_size - len(batch_results))  # Pad the results

        authenticity_results.extend(batch_results)
        total_tokens_processed += batch_tokens

        # Print status after processing each batch
        print(f"Processed inputs {i + 1} to {i + batch_size}, Results = {batch_results}")

        # Rate limiting: ensure only 2 requests per minute
        requests_made += 1
        if requests_made % max_requests_per_minute == 0:
            time.sleep(60)

    # Ensure the authenticity results list is the same length as the DataFrame
    if len(authenticity_results) < len(df):
        authenticity_results.extend([''] * (len(df) - len(authenticity_results)))

    # Assign results to new column
    df['authenticity'] = authenticity_results

    # Save the updated results back to the same file
    output_file = "processed_" + input_file
    df.to_csv(output_file, index=False, header=False)

    print("Processing complete. Results saved to", output_file)

# Example usage
input_file = 'mendeley_set.csv'  # Update with your file path
process_csv(input_file)

Response: response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=glm.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "Undetermined\nUndetermined\nFake\nUndetermined\nFake\nFake\nReal\nReal\nReal\nReal\nReal\nUndetermined\nReal\nReal\nUndetermined\nFake\nReal\nReal\nReal\nFake"
              }
            ],
            "role": "model"
          },
          "finish_reason": 1,
          "index": 0,
          "safety_ratings": [
            {
              "category": 9,
              "probability": 1,
              "blocked": false
            },
            {
              "category": 8,
              "probability": 1,
              "blocked": false
            },
            {
              "category": 7,
              "probability": 1,
              "blocked": false
            },
            {
              "category": 10,
              "probability": 1,
        